In [7]:
import numpy as np
import pickle
import sys
import torch
sys.path.append("../src")
from preprocess import new_grs, create_tensordata_new, convert_to_Data

In [2]:
data_path = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/ictal_1.pickle"
data = pickle.load(open(data_path, 'rb'))

#### Structure
List with entries of the form `graph = [A, NF, EF]`. Where `A`, `NF`, and `EF` are lists of length 4, 3, and 4 respectively. 

`A = [A0, A1, A2, A3]`, where :
-   `A0` = Ones, shape `(107,107)`.
-   `A1` = Correlation, shape `(107,107)`.  
-   `A2` = Coherence, shape `(107,107)`.
-   `A3` = Phase, shape `(107,107)`.

`NF = [NF0, NF1, NF2]`, where:

-  `NF0` = Ones, shape `(107,1)`.
-  `NF1` = Average Energy, shape `(107,1)`.
-  `NF2` = Band Energy, shape `(107,8)`.


`EF = [EF0, EF1, EF2, EF3]`, where:

-  `EF0` = Ones, shape `(107,107,1)`.
-  `EF1` = Correlation, shape `(107,107,1)`.
-  `EF2` = Coherence, shape `(107,107,1)`.
-  `EF3` = Phase, shape `(107,107,1)`.

All the information above has been (experimentally) confirmed by the above and Alan's documentation of `get_nf`, `get_adj`, and `get_ef` helper functions in his `load_data()` function, but should talk to Alan about confirming these details for absolute certainty.

In [3]:
print(f"A0 = E0 for first graph? {np.array_equal(data[0][0][0], data[0][2][0].reshape(107, 107))}.")
print(f"A1 = E1 for first graph? {np.array_equal(data[0][0][1], data[0][2][1].reshape(107, 107))}.")
print(f"A2 = E2 for first graph? {np.array_equal(data[0][0][2], data[0][2][2].reshape(107, 107))}.")
print(f"A3 = E3 for first graph? {np.array_equal(data[0][0][3], data[0][2][3].reshape(107, 107))}.")

A0 = E0 for first graph? True.
A1 = E1 for first graph? True.
A2 = E2 for first graph? True.
A3 = E3 for first graph? True.


#### Temporal Concatenation
For each run, we are going to concatenate the features in a larger list of `[preictal, ictal, postictal]`. We're then going to feed this into the `preprocess.ipynb` pipeline we've created (or just using one big patch). For simplicity we're going to use the most extensive graph representation possible by using:
-  `A` = None
-  `NF` = Average Energy and Band Energy, shape `(107,9)`.
-  `EF` = Correlation, Coherence, Phase, shape `(107, 107, 3)`.

for each patient.

We'll call this new list of graph representations `new_data`.

In [3]:
from preprocess import new_grs
new_data = new_grs(data, type="ictal", mode="binary")

Now we'll convert `new_data` to `pyg_data` so it is in tensor format.

We'll name this new list of torch tensors with entries `[[edge_index, x, edge_attr], y]` as `pyg_grs`.

In [4]:
pyg_grs = create_tensordata_new(num_nodes=107, data_list=new_data, complete=True, save=False, logdir=None)

After this we then convert the list of tensor `pyg_grs` to the PyG data structure `torch_geometric.data.Data`.

In [9]:
Data_list = convert_to_Data(pyg_grs, save=False, logdir=None)

In [12]:
print(Data_list[0])

Data(x=[107, 9], edge_index=[2, 11342], edge_attr=[11342, 3], y=1)


The rest is up to `preprocess.ipynb` to handle...